In [1]:
import pandas as pd
import plotly.express as px
import glob
import nibabel as nib

In [2]:
img = pd.read_csv('../../data/images/ADNI1_Complete_1Yr_1.5T_1_20_2022.csv')
img

# XXX_S_XXXX where the last 4 numbers correspond to the RID

FileNotFoundError: [Errno 2] No such file or directory: '../../data/images/ADNI1_Complete_1Yr_1.5T_1_20_2022.csv'

In [ ]:
image_files = glob.glob('../../data/raw/data/images/' + '**/*.nii', recursive=True)

# To access image filenames by their ID: key=image-id, value=filename.
image_map = {}
for filename in image_files:
    start_index = filename.rindex('_I')+1
    end_index = filename.rindex('.nii')
    image_id = filename[start_index:end_index]
    
    image_map[image_id] = filename

def process(image_id):
    return image_map.get(image_id)

img['filename'] = img['Image Data ID'].apply(process)
img.head()

In [ ]:
img['RID'] = img['Subject'].apply(lambda s: s[s.rindex('_')+1:])
img.head()

In [ ]:
img_columns = ['RID', 'Group', 'Visit', 'Acq Date', 'filename']
img = img[img_columns]
img

In [ ]:
#  https://dartbrains.org/content/Introduction_to_Neuroimaging_Data.html
# !conda install -c conda-forge nibabel -y # in the prompt

In [ ]:
# # convert a niftiimage to access it as numpy array
# img = nib.load(example_filename)
# data = img.get_fdata()

In [ ]:
# https://lukas-snoek.com/NI-edu/fMRI-introduction/week_1/python_for_mri.html
# Display an image, from the first row of the img df.
image_data = nib.load(img.iloc[0]['filename']).get_fdata()

In [ ]:
# No. of frames in z-dimension
image_data.shape[-1]

In [ ]:
px.imshow(image_data[:,:,80]) # px coordinates: all x-s, all y-s, 80. slide in z-dimension [1:157]?

In [ ]:
px.imshow(image_data[:,:,80])

In [ ]:
# https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

@interact
def show_image(x=75):
    return px.imshow(image_data[:,:,x])

In [ ]:
# Imports
import time
import numpy as np

from skimage import io

# vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = image_data.T
r, c = volume[0].shape # making a numpy.memmap()

# Define frames
import plotly.graph_objects as go
# nb_frames = image_data.shape[-1]
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='greys',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            'frame': {'duration': duration},
            'mode': 'immediate',
            'fromcurrent': True,
            'transition': {'duration': duration, 'easing': 'linear'},
        }

sliders = [
            {
                'pad': {'b': 10, 't': 60},
                'len': 0.9,
                'x': 0.1,
                'y': 0,
                'steps': [
                    {
                        'args': [[f.name], frame_args(0)],
                        'label': str(k),
                        'method': 'animate',
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                'buttons': [
                    {
                        'args': [None, frame_args(50)],
                        'label': '&#9654;', # play symbol
                        'method': 'animate',
                    },
                    {
                        'args': [[None], frame_args(0)],
                        'label': '&#9724;', # pause symbol
                        'method': 'animate',
                    },
                ],
                'direction': 'left',
                'pad': {'r': 10, 't': 70},
                'type': 'buttons',
                'x': 0.1,
                'y': 0,
            }
         ],
         sliders=sliders
)

fig.show()

### Sources

https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6

https://plotly.com/python/visualizing-mri-volume-slices/    
https://community.plotly.com/t/visualizing-mri-volume-slices-in-offline-mode/11004

"Memory-mapped files are used for accessing small segments of large files on disk, without reading the entire file into memory."
https://numpy.org/doc/stable/reference/generated/numpy.memmap.html

Maybe make images smaller, png?
https://github.com/FNNDSC/med2image
https://pypi.org/project/med2image/

https://lukas-snoek.com/NI-edu/fMRI-introduction/week_1/python_for_mri.html